In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import numpy as np
import plotly.graph_objects as go

In [2]:
data = pd.read_csv('data/AVG.csv', delimiter=';')
data.head()

,Players,Ratings,Minutes,Goals,Assists,Offsides,Passes,Accurate_Passes,Accurate_Passes_%,Crosses,Tackles,Total_Shots,Shots_on_Target,Blocked_Shots,Yellow_Cards,Red_Cards,Fouls_Committed,Clearances,Saves
0,D. Abramowicz,"7,10","90,00","0,20","0,20","0,40","28,00","21,60","76,99","1,80","0,40","1,20","0,20","0,00","0,20","0,00","1,00","5,40","0,00"
1,S. Kerk,"7,02","80,40","0,20","0,20","0,00","25,20","18,40","71,37","6,60","0,80","1,60","0,60","0,40","0,00","0,00","0,60","1,40","0,00"
2,J. Oliveira,"6,66","34,60","0,00","0,00","0,40","7,20","5,60","61,33","1,00","0,40","0,80","0,60","0,00","0,00","0,00","0,00","0,00","0,00"
3,A. Nguiamba,"5,43","24,20","0,00","0,00","0,00","11,40","9,40","62,78","0,20","1,60","0,00","0,00","0,00","0,00","0,00","0,20","0,40","0,00"
4,D. Zator,"2,72","13,20","0,00","0,00","0,00","7,40","5,60","29,68","0,60","0,80","0,00","0,00","0,00","0,00","0,00","0,00","0,60","0,00"


In [3]:
data = data.set_index('Players')
data.head()

,Ratings,Minutes,Goals,Assists,Offsides,Passes,Accurate_Passes,Accurate_Passes_%,Crosses,Tackles,Total_Shots,Shots_on_Target,Blocked_Shots,Yellow_Cards,Red_Cards,Fouls_Committed,Clearances,Saves
Players,,,,,,,,,,,,,,,,,,
D. Abramowicz,"7,10","90,00","0,20","0,20","0,40","28,00","21,60","76,99","1,80","0,40","1,20","0,20","0,00","0,20","0,00","1,00","5,40","0,00"
S. Kerk,"7,02","80,40","0,20","0,20","0,00","25,20","18,40","71,37","6,60","0,80","1,60","0,60","0,40","0,00","0,00","0,60","1,40","0,00"
J. Oliveira,"6,66","34,60","0,00","0,00","0,40","7,20","5,60","61,33","1,00","0,40","0,80","0,60","0,00","0,00","0,00","0,00","0,00","0,00"
A. Nguiamba,"5,43","24,20","0,00","0,00","0,00","11,40","9,40","62,78","0,20","1,60","0,00","0,00","0,00","0,00","0,00","0,20","0,40","0,00"
D. Zator,"2,72","13,20","0,00","0,00","0,00","7,40","5,60","29,68","0,60","0,80","0,00","0,00","0,00","0,00","0,00","0,00","0,60","0,00"


In [4]:
data = data.replace(',', '.', regex=True)
data = data.apply(pd.to_numeric, errors='coerce')

In [5]:
fig = go.Figure()

# List of metrics to plot
metrics = ['Ratings', 'Minutes', 'Goals', 'Assists', 'Offsides', 'Passes',
       'Accurate_Passes', 'Accurate_Passes_%', 'Crosses', 'Tackles',
       'Total_Shots', 'Shots_on_Target', 'Blocked_Shots', 'Yellow_Cards',
       'Red_Cards', 'Fouls_Committed', 'Clearances', 'Saves']

# Add traces for each metric
for metric in metrics:
    filtered_data = data[data[metrics] >  0]
    sorted_df = filtered_data.sort_values(by=metric, ascending=False)
    
    fig.add_trace(go.Bar(x=sorted_df.index, y=sorted_df[metric], name=metric))


# Define visibility matrix
n = len(metrics)
visibility_matrix = {
    "None": [False] * n,
    "AVG Rating": [metric in ['Ratings'] for metric in metrics],
    "Minutes": [metric in ['Minutes'] for metric in metrics],
    "Passes": [metric in ['Passes', 'Accurate_Passes', 'Accurate_Passes_%', 'Crosses'] for metric in metrics],
    "Attack": [metric in ['Goals', 'Assists', 'Total_Shots', 'Shots_on_Target', 'Shots_Inside_Box', 'Shots_Outside_Box'] for metric in metrics],
    "Defense": [metric in ['Tackles', 'Fouls_Drawn', 'Fouls_Committed', 'Saves'] for metric in metrics],
    "Cards": [metric in ['Yellow_Cards', 'Red_Cards'] for metric in metrics],
    "All": [True] * n
}

# Create buttons
buttons = [
    dict(label=label,
         method="update",
         args=[{"visible": visibility},
               {"title": f"Arka Match Stats: {label}"}])
    for label, visibility in visibility_matrix.items()
]

# Add dropdown to figure
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=1.1,
        y=1.15
    )],
    title="Arka Match Stats",
    xaxis_title="Match",
    yaxis_title="Value",
    barmode='group'
)

fig.show()
fig.write_html("Arka_stats_players_AVG.html")